# Getting the Git

In [6]:
import os
%matplotlib inline
import pandas as pd
import numpy as np
import scipy as sp
import timeit
import seaborn as sns
from joblib import Parallel, delayed
import multiprocessing
from multiprocessing import Pool
import matplotlib as mpl
import time
import csv
import math
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from time import sleep
import sys
import pickle
import codecs
import threading
os.environ['PATH'] += ";C:\\Program Files\\Git\\bin"

'''# ---------------- Old Toolbar Updated -------------------------#
toolbar_width = 40

# setup toolbar
sys.stdout.write("[%s]" % (" " * toolbar_width))
sys.stdout.flush()
sys.stdout.write("\b" * (toolbar_width+1)) 
'''

def progbar(i,j):
    sys.stdout.write('\r')
    sys.stdout.write("%.2f%%" % round((i / j) * 100,2))
    sys.stdout.flush()
    sleep(0.001)

# Opening the full data

# Fuzzy matching on firm names

I use the fuzzywuzzy package and run through each firm name, comparing it to the remaining 5.6 million in the dataset. 

Each of the firm names is compared against a master list which is stripped of any non-string types (although I'm not sure why any would be in there, there seem to be a few.) I also drop duplicates but, again, I don't think there should be too many in there, given all the work already conducted using Stata.

In [7]:
cleanedrec = pickle.load(open('cleanedrecs.p','rb'))
cleanedrec = list(set(cleanedrec))
print(len(cleanedrec))
cleanedrec[:5]

4820948


['',
 'wilshire state bank',
 'weber construction',
 'aptos chamber of commerce',
 'dice photography']


Confirmed that there are no duplicates in the list. 

Now we're off to the races. This is going to take a goddamned long time so smoke 'em if you got 'em. We go through every firm name in the full list of 5.6m and compare the string to all of the others using the fuzzywuzzy method "process". 

Whoa...too long. I'm going to try and run this on the cluster. No sense in gumming up my machine.

In [8]:
dictter = dict(zip(range(569),range(0,6 * 10 ** 6,10 ** 4)))
start = dictter[50] #dictter[int(sys.argv[1])]
end = dictter[50+1] #dictter[int(sys.argv[1]+1)]
print(start,end)
end = end/1000 # Comment this line out to run the 10k chunks.
end

500000 510000


510.0

# Threading test

I need to figure out how to extract content from these threads. Right now, I'm only messing around with the thread workers themselves but surely the function output is stored somewhere within them? I have also messed around with multiprocessing and the joblib packages but these don't seem to work within Jupyter (and require a cluge to work in Windows at all via command line calls).

# A basic loop for pairing

Before I get started, I want to do some basic text cleaning to convert things like "corporation", "incorporated", "city", "university", etc. (Note that this is raw'ed out after I successfully ran it because it takes for goddamned ever.)

# NEED TO SPEED THINGS UP. 

Some searching led me to https://github.com/schollz/string_matching which uses a dictionary to dramatically reduce the number of words in the full list that are relevant for the string matching algorithm. I'm going to try and hack it into my process. (Right now, the process of 9 weeks of 300 jobs running constantly seems a little insane.) 

## Setting up pickle files via setup.py


First I need to recreate the functions since I'm not sure how to import from a folder that's locally stored.

In [27]:
def generateSearchableHashFromList(listOfStrings):
    sHash = {}
    for string in listOfStrings:
        for i in range(0,len(string)-2):
            doublet = string[i:i+3].lower()
            if doublet not in sHash:
                sHash[doublet] = []
            sHash[doublet].append(string)
    return sHash

In [28]:
import string_matching
from time import time 

listchunk = cleanedrec

# Make a list of words, save it as "defaultList.p"
tt = time()
defaultList = []
for line in listchunk:
        defaultList.append(line.strip())
pickle.dump(defaultList,open('defaultList.p','wb'))
print ('dumping defaultList took ' + str(time()-tt))


# Generate the searchable hash, save it as "defaultHash.p"
tt = time()
defaultHash = generateSearchableHashFromList(defaultList)
pickle.dump(defaultHash,open('defaultHash.p','wb'))
print ('dumping defaultHash took ' + str(time()-tt))


dumping defaultList took 5.431874513626099
dumping defaultHash took 73.74902558326721


In [9]:
defaultList = pickle.load(open('defaultList.p','rb'))
defaultHash = pickle.load(open('defaultHash.p','rb'))

### This is all old but I think it will be useful to keep around. 

So the tutorial provided by ja boy schollz is cool and all but I think I'm going to keep my pairings function and then just run it on the subset of data associated with the best matches according to the hash approach. 

## Try a short example

This section recreates the content of the searchThroughList function

This section recreates the searchThroughHash function. This is the part I want to keep, but instead of running the searchThroughList() function at the end, I want to run my own extractBests function borrowed from fuzzywuzzy.

In [24]:
def pairs(searchString,stringList):
    return [searchString,[x[0] for x in process.extractBests(searchString,stringList,scorer=fuzz.token_set_ratio,score_cutoff=90)]]

In [20]:
def searchThroughHash(searchString,sHash=defaultHash):
    if len(sHash)==0:
        return "You need to save a hash to defaultHash.p if you want to load one automatically!"
    searchString = searchString.lower()
    possibleStrings = []
    for i in range(0,len(searchString)-2):
        doublet = searchString[i:i+3]
        if doublet in sHash:
            possibleStrings += sHash[doublet]
    c = Counter(possibleStrings)
    mostPossible = []
    for p in c.most_common(1000):
        mostPossible.append(p[0])
    return pairs(searchString,mostPossible)

In [29]:
searchThroughHash(cleanedrec[1],defaultHash)

['wilshire state bank', ['wilshire state bank', 'c state bank']]

In [22]:
from collections import Counter

In [30]:
searchString=cleanedrec[1].lower()
possibleStrings=[]
counter = 1
for i in range(0,len(searchString)-2):
    progbar(counter,len(searchString)-1)
    counter+=1
    doublet = searchString[i:i+3]
    if doublet in defaultHash:
        possibleStrings +=defaultHash[doublet]
possibleStrings[:5]
c = Counter(possibleStrings)
mostPossible = []
for p in c.most_common(1000):
    mostPossible.append(p[0])
mostPossible[:5]

96.15%

['Gerald J Katherine I Kurian',
 'Katherine Marcell Hazen Rev Living Trust Katherine Kirk Hazen T',
 'Gerald Katherine Goss',
 'Catherine Square Catherine Square',
 'Katherine A Ronald J Kochevar']

In [ ]:
def testStringMatchin(string):
    print ("trying with " + string)
    tt = time()
    print ('trying list look-up took ' + str(time()-tt))
    print ("\n")

    print ("searching through entire list...")
    tt = time()
    result = searchThroughList(string)
    print ("Best match: " + result)
    print ('searching through entire list took ' + str(time()-tt))
    print ("\n")

    print ("searching through hashed list...")
    tt = time()
    result = searchThroughHash(string)
    print ("Best match: " + result)
    print ('searching through hashed list took ' + str(time()-tt))

In [ ]:
#result = searchThroughList("pillow")
result

In [ ]:
test_string_matching(pillow)

This first pass produces many matches that it shouldn't and also matches a single firm name to multiple other names. I need to clean it up so that I have (1) each firm is associated with only one group of firm names and (2) for the survey, I'm only looking at firms that are matched to others. (I will go back to the original pairings code when creating the manual crosswalk.)

In [ ]:
# Don't match firm names to themselves
noself = []
for j in range(len(pairings)):
    noself.append([pairings[j][0],[i for i in pairings[j][1] if pairings[j][0] != i]])

# For the survey, only get those firm names that were matched with at least one other
survey1 = [noself[i] for i in range(len(noself)) if len(noself[i][1]) > 0]
print(survey1[0])

# However, this produces many clearly false matches. The token_set_ratio function is too greedy so I apply a different function to the paired data.
lessgreedy = []
for j in range(len(survey1)):
    lessgreedy.append([survey1[j][0],[x[0] for x in process.extractBests(survey1[j][0],survey1[j][1],scorer = fuzz.token_sort_ratio,score_cutoff=70)]])
print(lessgreedy[0])

# Again, for the survey only use matches that were paired with at least one other
survey2 = [lessgreedy[i] for i in range(len(lessgreedy)) if len(lessgreedy[i][1]) > 0]

# Finally, for the survey I want to remove all instances in which the 'anchor' and the 'pair(s)' simply flip themselves.
forsurvey=[]
for i in range(len(survey2)-1):
    if len(survey2[i][1]) == 1:
        j = i+1
        if [survey2[i][0],survey2[i][1][0]] != [survey2[j][1][0],survey2[j][0]]:
            forsurvey.append(survey2[i])
    else:
        forsurvey.append(survey2[i])
forsurvey[:10]

However, we still find that an anchor's pair may itself have other pairs that aren't just the inverse. For example, \#1 CLEANERS is paired with \$1.25 CLEANERS but this name is also associated with \$1.25 DRY-CLEANERS. Ideally, I want to take all names that are associated with multiple pairs and run a sophisticated algorithm that will identify the best possible pairing (i.e., \$1.25 DRY-CLEANERS, not \#1 CLEANERS.

In [ ]:
# We know that anchors can only show up once in the full list of anchors.
anchors = [forsurvey[x][0] for x in range(len(forsurvey))]
anchors[:10]

# We are interested in whether a paired name shows up elsewhere as (1) its own anchor (i.e., after having stripped inverses)
# or (2) as a pair to a different anchor.
pairs=[]
for j in range(len(forsurvey)):
    for i in range(len(forsurvey[j][1])):
        pairs.append(forsurvey[j][1][i])
pairs[:10]

# For the first question, if a pair shows up as its own anchor, this is the same as an anchor showing up in the pairs
anchordupes = []
for j in range(len(forsurvey)):
    temp = []
    for i in range(len(forsurvey)):
        if forsurvey[j][0] in forsurvey[i][1]:
            for k in range(len(forsurvey[i][1])):
                if forsurvey[j][0] != forsurvey[i][1][k]:
                    temp.append(forsurvey[i][1][k])
    anchordupes.append([forsurvey[j][0],list(set(temp))])
        
# We now clean the anchor duplicates and choose the best possible match for those that remain
anchordupes1 = [anchordupes[i] for i in range(len(anchordupes)) if len(anchordupes[i][1]) > 0]

# For the second question, we simple repeat the above but go through all pairs that aren't the self
forsurvey[0]
pairs = list(set(pairs))

pairsdupes = []
for j in range(len(pairs)):
    temp=[]
    for i in range(len(forsurvey)):
        if pairs[j] in forsurvey[i][1]:
            temp.append(forsurvey[i][0])
    pairsdupes.append([pairs[j],temp])

pairsdupes1 = [pairsdupes[i] for i in range(len(pairsdupes)) if len(pairsdupes[i][1]) > 0]
pairsdupes1

# Finally, I merge these together by combining all matches for each unique anchor
fullanchors = [anchordupes1[x][0] for x in range(len(anchordupes1))]
fullanchors = list(set(fullanchors))
fullanchors

fullpairs = [pairsdupes1[x][0] for x in range(len(pairsdupes1))]
fullpairs = list(set(fullpairs))
fullpairs

full = fullanchors + fullpairs
full = list(set(full))
full

pulls = []
for i in range(len(full)):
    temp1=[]
    for j in range(len(anchordupes1)):
        if full[i] == anchordupes1[j][0]:
            temp1.append(anchordupes1[j][1])
    temp2 = []
    for j in range(len(pairsdupes1)):
        if full[i] == pairsdupes1[j][0]:
            temp2.append(pairsdupes1[j][1])
    if len(temp1) > 0 or len(temp2) > 0:
        all = []
        if len(temp1) > 0:
            for x in range(len(temp1)):
                all.append(temp1[x])
        if len(temp2) > 0:
            for y in range(len(temp2)):
                all.append(temp2[y])
        all2 = []
        for z in range(len(all)):
            for a in range(len(all[z])):
                all2.append(all[z][a])
    pulls.append([full[i],list(set(all2))])


# Jesus...almost there. I think the last step is very similar to the first one. I need to aggregate all daisy-chained 
# pairs into a single grouping.
finaltemp = []
for i in range(len(pulls)):
    temp=[]
    if len(pulls[i][1]) > 0:
        for j in range(len(pulls[i][1])):
            for k in range(len(pulls)):
                if pulls[i][1][j] == pulls[k][0] and i != k:
                    #print('pair %s in pull %s is at %s' %(j,i,k))
                    for l in range(len(pulls[k][1])):
                        if pulls[k][1][l] != pulls[i][0]:
                            temp.append(pulls[k][1][l])
                    if k > i:
                        pulls[k][0] = "DUPES"
        finaltemp.append([pulls[i][0],list(set(temp))])

final = []
for i in range(len(finaltemp)):
    if finaltemp[i][0] != 'DUPES' and len(finaltemp[i][1])>0:
        final.append(finaltemp[i])

final

Jesus. If Drew every saw that mess, he would kill me. I just used 200 lines of code to accomplish something that I reckon should be pretty simple. But fuck it. I am building intuition for pyton. 

And what exactly did I accomplish? I'm not entirely sure. I've created a list of firm names that aggregates all the possible pairings that have exceeded the two thresholds I've put them through via the fuzzywuzzy tools. 

# Write a .txt file in the format for importing to Qualtrics

The code below can be converted to illustrate the process by which I create the .txt files that are formatted to be imported into Qualtrics. I don't run the code here though because it would take a long time.

In [ ]:
maxs = int(math.ceil(len(final) / 100))
print(maxs)
length = len(final)

start = 0
counter = 0

for z in range(1,maxs+1):
    end = min(z*100,length)
    submask = final[start:end]
    start = end+1
    survey_file = open("range%s_survey_test_%s" %(1,z) +".txt","w")
    survey_file.write("[[AdvancedFormat]] \n\n")
    for j in range(0,len(submask),10):
        survey_file.write("[[Block: Firm Names - %s" % counter + "]]\n")
        for k in range(j,min(j+10,len(submask))):
            survey_file.write(" [[Question:MC:MultipleAnswer:Vertical]] \n \
            %s" % submask[k][0].upper() + "\n\n")
            survey_file.write("  [[AdvancedChoices]] \n")
            for i in range(0,len(submask[k][1])):
                survey_file.write("    [[Choice: %s" % i +"]] \n \
                %s" % submask[k][1][i].upper() +'\n')
            survey_file.write("\n")
        progbar(counter,maxs*10)
        counter += 1
    survey_file.close()

Now, for my final act, I will assign unique ids to every firm name in the dataset. This means I need to reincorporate the individual values assembled above. 

In [ ]:
indivs = [pairings[x][0] for x in range(len(pairings))]
finalanchors = [final[x][0] for x in range(len(final))]
all = []
for i in range(len(indivs)):
    if indivs[i] in finalanchors:
        for j in range(len(finalanchors)):
            if indivs[i] == final[j][0]:
                all.append(final[j])
    else:
        all.append([indivs[i],[]])
all

In [ ]:
crosswalk1 = []
bisids=[]
for i in range(len(all)):
    crosswalk1.append(all[i][0])
    bisids.append("bid"+str(i))
    for j in range(len(all[i][1])):
        crosswalk1.append(all[i][1][j])
        bisids.append("bid"+str(i))

crosswalk = pd.DataFrame({'cw1' : crosswalk1,'bid':bisids})
print(forsurvey[0])
print(forsurvey[0][1])
#print(mask[23][1][0][0])

#inds = []
#for x in range(len(forsurvey)):
#    for i in range(len(forsurvey[x][1])):
#         if mask[23][1][0] in forsurvey[x][1][i]:
#                inds.append(x)
#mask[inds[1]]
#inds

crosswalk

In [ ]:
args = 1
name = str('test%s' % args + '.csv')
header = ['bid','cw1']
crosswalk.to_csv(name,sep=',',columns=header,index=False)

In [ ]:
%%bash
git add fec-string-prep.ipynb
git status
git commit -m "Further tinkering with the algorithm and attempting to get things parallelized."
git push
git status

In [ ]:
%%bash
git status